To use a running delayed start service:

1. Register service (here use of service) on Service Store

2. Have generic adapter instance running

3. Using the generic adapter...  
	3.1 ... log in to service store and get hash for service you want to use  
	3.2 ... register knowledge base A  
	3.3 ... create smart connector A  
	3.4 ... register knowledge interaction  
	3.5 ... execute ask knowledge interaction  

5. Format 'complete set' to useable forecast time series

**This is what the BOSCH (and other appliances) SSA will be doing!**

To run Generic Adapter docker container:  
docker run -p 9001:9001 -p 9090:9090 --name Generic_Adapter docker-registry.inesctec.pt/interconnect/generic-adapter:2.1-whirlpool 

With different ports:  
docker run -p 9002:9001 -p 9091:9090 --name Generic_Adapter_2 docker-registry.inesctec.pt/interconnect/generic-adapter:2.1-whirlpool

In [4]:
import os
import requests
import json
import datetime
from interconnect_credentials import username, password, username_interconnect, password_interconnect, graph_pattern_prefixes

ModuleNotFoundError: No module named 'interconnect_credentials'

In [ ]:
files_and_names = {'bsh': {'service_name': 'BSH Delayed Start',
                          'argument_graph_pattern_file': "bsh_powerprofile_graphpattern.txt",
                          'result_graph_pattern_file': "non-existing-file.txt",
                          'binding_set_file': 'bsh_powerprofile_bindingset.json'},
                   'miele': {'service_name': 'Miele Delayed Start',
                          'argument_graph_pattern_file': "bsh_powerprofile_graphpattern.txt",
                          'result_graph_pattern_file': "non-existing-file.txt",
                          'binding_set_file': 'bsh_powerprofile_bindingset.json'},
                  'whirlpool': {'service_name': 'Whirlpool Delayed Start',
                          'argument_graph_pattern_file': "whirlpool_powerprofile_argument_graphpattern.txt",
                          'result_graph_pattern_file': "whirlpool_powerprofile_result_graphpattern.txt",
                          'binding_set_file': './whirlpool_powerprofile_bindingset.json'}}

In [3]:
supplier = 'miele'
base_url = 'http://localhost:9090'
service = files_and_names[supplier]['service_name']
argument_graph_pattern_file = files_and_names[supplier]['argument_graph_pattern_file']
result_graph_pattern_file = files_and_names[supplier]['result_graph_pattern_file']
binding_set_file = files_and_names[supplier]['binding_set_file']

NameError: name 'files_and_names' is not defined

**Check if the Generic Adapter instance is up and running**

In [2]:
response = requests.get(base_url)
response.text

NameError: name 'base_url' is not defined

# Login to the Service Store (https://store.interconnectproject.eu/backend)

In [6]:
response = requests.post(base_url + '/servicestore/login', json={"email": username_interconnect,
                                                                "password": password_interconnect})
access_token = response.json().get('access_token')
header = {'Authorization': 'Bearer ' + access_token}

# Get all services offered by OpenMotics

In [12]:
response = requests.get(base_url + '/servicestore/service-list')
response = response.json()
[d['name'] for d in response] 

['Interconnect DSO Interface',
 'Altherma Homelab',
 'ReFlex',
 'Hyrde Production',
 'SmartMeterData',
 'INESCTEC Thermovault SSA',
 'Live Data Service new',
 'Answer Measure test',
 'whirlpool_appliances_service_ask',
 'Ask measure test',
 'INESC DSO Interface Service',
 'Greek Pilot EMS-Spine React Interaction Service',
 'Measurement of flexibility data',
 'SENSI SE Meter Reading new KE',
 'UDX',
 'Cosmote_User_Authentication_Service',
 'KB1PostService',
 'VFlexData',
 'RML Mapping POST service',
 'Hyrde Staging',
 'Hyrde First Test Answer',
 'InterconnectTestCGI',
 'VITO Configuration Service',
 'Cosmote_historical_data_answer',
 'Whirlpool Delayed Start',
 'EMS',
 'TesteService',
 'INESC Whirlpool Service',
 'Linc Live Data',
 'Sensinov DSO interface new KE',
 'WINGS recommendation service',
 'EV Platform Dynamic Tariff Retrieval',
 'SENSI SE Cooling Reading new KE',
 'Post user feedback',
 'Tutorial Test KE Handle',
 'INESC Bosch Service',
 'KB1ReactService',
 'AnswerService',
 'T

In [8]:
response

[{'serviceID': 2053747,
  'active': True,
  'visible': True,
  'registerDate': '2023-04-11T12:44:17.851893',
  'expirationDate': '2021-03-20T00:00:00.000+00:00',
  'description': 'Service with all DSO Interface interactions',
  'name': 'Interconnect DSO Interface',
  'type': 'Other',
  'platform': 'E-REDES',
  'ontology': 'SAREF',
  'url': 'https://ke-pt.interconnectproject.eu/rest/',
  'ontologyVersion': 1,
  'certificationLevel': 1,
  'hash': 'bebda4180c06bcade9b1fdd070789ed471baef536602fafd446a2b8716c155ad559b6aa446499f5b4cec54e2ec5d9bdaea0b68c7a55d4b0b5efede327354d47b'},
 {'serviceID': 1882186,
  'active': True,
  'visible': True,
  'registerDate': '2023-03-16T09:53:43.607776',
  'expirationDate': '2021-03-20T00:00:00.000+00:00',
  'description': 'Altherma Homelab',
  'name': 'Altherma Homelab',
  'type': '',
  'platform': 'Daikin',
  'ontology': 'SAREF',
  'url': 'localhost',
  'ontologyVersion': 1,
  'certificationLevel': 1,
  'hash': '5ec4bbfc126401012d9c2f83c1b86b2efc91abd1b07d

# Register a Knowledge Base for the PV Forecaster user

In [13]:
hash_ = [d['hash'] for d in response if d['name'] == service]

In [14]:
body = {"hash": hash_[0], 
        "language": "Python", 
        "transport": "HTTP"}
if supplier == 'whirlpool':
    body.update({"optionalKnowledgeEngine": f"http://knowledge-engine:8280/rest"})

response = requests.post(base_url + '/servicestore/adapter/register', json=body)

knowledgebase_id = response.text

# Register a Smart Connector coupled to the Knowledge Base

In [15]:
body = {"knowledgeBaseId": knowledgebase_id,
        "knowledgeBaseName": "delayedstart_service_user",
        "knowledgeBaseDescription": "description of the delayedstart_service_user knowledge base",
        "leaseRenewalTime": 3600}

response = requests.post(base_url + '/smartconnector/create', json=body)

# Check if the Smart Connector was registered

In [16]:
response = requests.get(base_url + '/smartconnector/get-all-or-one',
                        headers={"KnowledgeBaseId": knowledgebase_id})
response.json()

[{'knowledgeBaseId': 'https://ke.interconnectproject.eu/rest/adapter/c084cde1937fb25794e672e5b0a5618e97b57007e07e6cb6876000da945a40b137e3917346732234664289950dbe3d685809453759c1109f3daffa9561942f38',
  'knowledgeBaseName': 'delayedstart_service_user',
  'knowledgeBaseDescription': 'description of the delayedstart_service_user knowledge base',
  'leaseRenewalTime': 3600,
  'reasonerEnabled': False}]

# Register Knowledge Interaction for use of the start delay service

In [33]:
f = open(argument_graph_pattern_file, "r")
argument_graph_pattern = ' '.join([l.replace('\n','') for l in f.readlines()])

try:
    f = open(result_graph_pattern_file, "r")
    result_graph_pattern = ' '.join([l.replace('\n','') for l in f.readlines()])
except FileNotFoundError:
    result_graph_pattern = None

f = open("graph_pattern_prefixes.json")
graph_pattern_prefixes = json.load(f)

In [34]:
argument_graph_pattern

'?powerSequence <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://saref.etsi.org/saref4ener/PowerSequence> . ?powerSequence <https://saref.etsi.org/saref4ener/sequenceID> ?sequenceID . ?powerSequence <https://saref.etsi.org/core/hasState> ?state . ?powerSequence <http://ontology.tno.nl/interconnect/topology#hasTopologicalAssociation> ?associatedDevice . ?powerSequence <https://saref.etsi.org/saref4ener/activeSlotNumber> ?activeSlotNumber . ?powerSequence <https://saref.etsi.org/saref4ener/sequenceRemoteControllable> ?sequenceRemoteControllable . ?powerSequence <https://saref.etsi.org/saref4ener/StartTime> ?startTime . ?powerSequence <https://saref.etsi.org/saref4ener/EndTime> ?endTime . ?powerSequence <https://saref.etsi.org/saref4ener/EarliestStartTime> ?earliestStartTime . ?powerSequence <https://saref.etsi.org/saref4ener/LatestEndTime> ?latestEndTime . ?powerSequence <https://saref.etsi.org/saref4ener/isPausable> ?isPausable . ?powerSequence <https://saref.etsi.org/saref4en

In [19]:
result_graph_pattern

In [41]:
body = {"knowledgeInteractionType": "PostKnowledgeInteraction",
        "knowledgeInteractionName": "react-ki",
        "communicativeAct": {
            "requiredPurposes": ["https://www.tno.nl/energy/ontology/interconnect#InformPurpose"],
            "satisfiedPurposes": ["https://www.tno.nl/energy/ontology/interconnect#InformPurpose"]},
        "argumentGraphPattern": argument_graph_pattern,
        "resultGraphPattern": result_graph_pattern,
        "prefixes": graph_pattern_prefixes}

response = requests.post(base_url + '/smartconnector/ki/register-post-react', 
                         headers={"KnowledgeBaseId": knowledgebase_id},
                         json=body)

knowledgeinteraction_id = response.json().get('knowledgeInteractionId')

In [42]:
response.json()

{'knowledgeInteractionId': 'https://ke.interconnectproject.eu/rest/adapter/c084cde1937fb25794e672e5b0a5618e97b57007e07e6cb6876000da945a40b137e3917346732234664289950dbe3d685809453759c1109f3daffa9561942f38/interaction/react-ki'}

In [43]:
body

{'knowledgeInteractionType': 'PostKnowledgeInteraction',
 'knowledgeInteractionName': 'react-ki',
 'communicativeAct': {'requiredPurposes': ['https://www.tno.nl/energy/ontology/interconnect#InformPurpose'],
  'satisfiedPurposes': ['https://www.tno.nl/energy/ontology/interconnect#InformPurpose']},
 'argumentGraphPattern': '?powerSequence <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://saref.etsi.org/saref4ener/PowerSequence> . ?powerSequence <https://saref.etsi.org/saref4ener/sequenceID> ?sequenceID . ?powerSequence <https://saref.etsi.org/core/hasState> ?state . ?powerSequence <http://ontology.tno.nl/interconnect/topology#hasTopologicalAssociation> ?associatedDevice . ?powerSequence <https://saref.etsi.org/saref4ener/activeSlotNumber> ?activeSlotNumber . ?powerSequence <https://saref.etsi.org/saref4ener/sequenceRemoteControllable> ?sequenceRemoteControllable . ?powerSequence <https://saref.etsi.org/saref4ener/StartTime> ?startTime . ?powerSequence <https://saref.etsi.org/sar

In [18]:
#response = requests.delete(base_url + '/smartconnector/delete',
#                           headers={"KnowledgeBaseId": knowledgebase_id})

In [19]:
#response = requests.delete(base_url + '/smartconnector/ki/unregister',
#                           headers={"KnowledgeBaseId": knowledgebase_id, 
#                                   "KnowledgeInteractionId": knowledgeinteraction_id})

# Send ask or post

In [44]:
f = open(binding_set_file)
binding_set = json.load(f)

In [45]:
if isinstance(binding_set, list):
    for item in binding_set:
        append = '^^<http://www.w3.org/2001/XMLSchema#dateTime>' if '^^' in item['earliestStartTime'] else ''
        item['earliestStartTime'] = f'"{datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S+0000")}"{append}'
        item['latestEndTime'] = f'"{(datetime.datetime.now() + datetime.timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M:%S+0000")}"{append}'
else:
    append = '^^<http://www.w3.org/2001/XMLSchema#dateTime>' if '^^' in item['earliestStartTime'] else ''
    binding_set['earliestStartTime'] = f'"{datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S+0000")}"{append}'
    binding_set['latestEndTime'] = f'"{(datetime.datetime.now() + datetime.timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M:%S+0000")}"{append}'
    binding_set = [binding_set]

In [46]:
binding_set[0].keys()

dict_keys(['powerSequence', 'valueExpectedValue_1', 'deviceId', 'associatedDevice', 'sequenceRemoteControllable', 'slot_1', 'powerMax_1', 'earliestStartTime', 'isPausable', 'slotNumber_1', 'activeSlotNumber', 'startTime', 'valueMin_1', 'state', 'measurementExpectedValue_1', 'valueMax_1', 'measurementMax_1', 'latestEndTime', 'powerExpectedValue_1', 'defaultDuration_1', 'sequenceID', 'measurementMin_1', 'isStoppable', 'powerMin_1', 'endTime'])

In [47]:
response = requests.post(base_url + '/smartconnector/post', 
                         headers={"KnowledgeBaseId": knowledgebase_id,
                                  "KnowledgeInteractionId": knowledgeinteraction_id},
                         json=binding_set)
complete_set = response.json().get('resultBindingSet')

In [29]:
response.json()

{'resultBindingSet': [],
 'exchangeInfo': [{'initiator': 'knowledgeBase',
   'knowledgeBaseId': 'https://ke.interconnectproject.eu/rest/adapter/b154fb2a122733201c2ea2361243bd3234b92ff4627f6f0804ddc09f33484e71ed4796b92e60f739f09213cbeb242ce6692648164d0aa2608a2158594e95508b',
   'knowledgeInteractionId': 'https://ke.interconnectproject.eu/rest/adapter/b154fb2a122733201c2ea2361243bd3234b92ff4627f6f0804ddc09f33484e71ed4796b92e60f739f09213cbeb242ce6692648164d0aa2608a2158594e95508b/interaction/MielePowerProfileKI',
   'exchangeStart': '2023-08-16T07:27:16.834+00:00',
   'exchangeEnd': '2023-08-16T07:27:17.345+00:00',
   'status': 'SUCCEEDED',
   'failedMessage': None,
   'argumentBindingSet': [{'powerSequence': '<http://example.org/spine-ssa/devices/SIEMENS-WT47XM40-68A40E31F2EA/powerSequences/188040199>',
     'valueMax_1': '"970"',
     'measurementMax_1': '<https://example/slot/measurementMax/10>',
     'latestEndTime': '"2023-08-16T21:15:54+0000"',
     'valueExpectedValue_1': '"500"',
 

In [53]:
complete_set

[{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>',
  'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>',
  'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>'}]

Try using RDF binding set

In [79]:
f = open('./startdelay_bindingset_short_rdf.txt', "r")
rdfbinding_set = f.read()

In [80]:
rdfbinding_set

'<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/1671100271> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://saref.etsi.org/saref4ener/PowerSequence> .\n<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/1671100271> <https://saref.etsi.org/saref4ener/sequenceID> "1671100271" .\n<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/1671100271> <https://saref.etsi.org/core/hasState> "scheduled" .\n<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/1671100271> <https://saref.etsi.org/saref4ener/activeSlotNumber> "0" .\n<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/1671100271> <https://saref.etsi.org/saref4ener/sequenceRemoteControllable> "true" .\n<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/1671100271> <https://saref.etsi.org/saref4ener/StartTime> "2022-12-15T11:01:10.755029Z" .\n<http://example.org/spine-ssa/devices/012100518398009540/powerS

In [81]:
response = requests.post(base_url + '/smartconnector/RDF/post', 
                         headers={"KnowledgeBaseId": knowledgebase_id,
                                  "KnowledgeInteractionId": knowledgeinteraction_id},
                         json={"RDFBindingSet": rdfbinding_set})
complete_set = response.json().get('bindingSet')

# Format binding set

In [2]:
import datetime
from dateutil.parser import parse

In [3]:
def parse_literal(literal, parse_as=None):
    """
    Example literals:
    '"2021-12-03T15:01:27+01:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>'
    '"38.709880737151195"^^<http://www.w3.org/2001/XMLSchema#decimal>'

    returns
    DateTime object for dateTime literals
    float object for decimal literals
    """
    uri_parse_mapping = {'datetime': '<http://www.w3.org/2001/XMLSchema#dateTime>',
                         'time': '<http://www.w3.org/2001/XMLSchema#time>',
                         'float': '<http://www.w3.org/2001/XMLSchema#decimal>',
                         'int': '<http://www.w3.org/2001/XMLSchema#int>',
                         'bool': '<http://www.w3.org/2001/XMLSchema#boolean>',
                        'str': '<http://www.w3.org/2001/XMLSchema#string>'}
    
    def parse_time(t):
        time_split = t.split(':')
        return datetime.timedelta(hours=int(time_split[0]),
                                  minutes=int(time_split[1]),
                                  seconds=int(time_split[2]))

    value_schema = literal.split('^^')
    if parse_as is None:
        uri = value_schema[1]
    else:
        uri = uri_parse_mapping.get(parse_as)
    value = value_schema[0].replace("'", '').replace('"', '')
    if uri == '<http://www.w3.org/2001/XMLSchema#dateTime>':
        value = parse(value)
    elif uri == '<http://www.w3.org/2001/XMLSchema#time>':
        value = parse_time(value)
    elif uri == '<http://www.w3.org/2001/XMLSchema#decimal>':
        try:
            value = float(value)
        except ValueError:
            value = None
    elif uri == '<http://www.w3.org/2001/XMLSchema#int>' or \
        uri == '<http://www.w3.org/2001/XMLSchema#unsignedInt>' or \
        uri == '<http://www.w3.org/2001/XMLSchema#integer>':
        try:
            value = int(value)
        except ValueError:
            value = None
    elif uri == '<http://www.w3.org/2001/XMLSchema#boolean>':
        try:
            value = bool(value)
        except ValueError:
            value = None
    elif uri == '<http://www.w3.org/2001/XMLSchema#string>':
        try:
            value = str(value)
        except ValueError:
            value = None
    else:
        raise Exception(f"Unit iri {uri} is not known!")

    return value

## BSH/Miele

In [100]:
binding_set = [{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence>', 
                'valueMax_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 
                'measurementMax_1': '<https://saref.etsi.org/core/Measurement>', 
                'latestEndTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 
                'valueExpectedValue_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 
                'powerExpectedValue_1': '<https://saref.etsi.org/saref4ener/PowerExpectedValue>', 
                'defaultDuration_1': "'00:23:00'^^<http://www.w3.org/2001/XMLSchema#time>", 
                'sequenceID': "'1'^^<http://www.w3.org/2001/XMLSchema#int>", 
                'sequenceRemoteControllable': "'false'^^<http://www.w3.org/2001/XMLSchema#boolean>", 
                'measurementMin_1': '<https://saref.etsi.org/core/Measurement>', 
                'powerMax_1': '<https://saref.etsi.org/saref4ener/PowerMax>', 
                'slot_1': '<https://saref.etsi.org/saref4ener/Slot>', 
                'isStoppable': "'false'^^<http://www.w3.org/2001/XMLSchema#boolean>", 
                'earliestStartTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 
                'slotNumber_1': "'1'^^<http://www.w3.org/2001/XMLSchema#int>", 
                'activeSlotNumber': "'1'^^<http://www.w3.org/2001/XMLSchema#int>", 
                'powerMin_1': '<https://saref.etsi.org/saref4ener/PowerMin>', 
                'startTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 
                'valueMin_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 
                'endTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 
                'state': "'scheduled'^^<http://www.w3.org/2001/XMLSchema#string>", 
                'measurementExpectedValue_1': '<https://saref.etsi.org/core/Measurement>'}]

In [111]:
power_sequence = {}
for datapoint in binding_set:  # In fact, only one data point is expected
    # Get values that are general to the sequence, not for a specific slot
    power_sequence.update({k: parse_literal(datapoint[k]) for k, v in datapoint.items() if
                           not any(c.isdigit() for c in k) and
                           'http://www.w3.org/2001/XMLSchema#' in v})
    power_sequence[f'sequence'] = {}
    power_sequence[f'sequence'][datetime.timedelta(0)] = parse_literal(datapoint[f'valueExpectedValue_1'])
    print(power_sequence)
    n = 2
    print(datapoint)
    while datapoint.get(f'slot_{n}') is not None:
        power_sequence[f'sequence'][max(power_sequence['sequence'].keys()) +
                                    parse_literal(datapoint[f'defaultDuration_{n-1}'])] = \
            parse_literal(datapoint[f'valueExpectedValue_{n}'])
        # only the expected values are used; future developments can possibly include the minimum and maximum
        # expected values
        n += 1

    power_sequence[f'sequence'][max(power_sequence['sequence'].keys()) +
                                parse_literal(datapoint[f'defaultDuration_{n-1}'])] = 0.0
    print(power_sequence)

{'latestEndTime': datetime.datetime(2021, 12, 8, 13, 6, 48, tzinfo=tzoffset(None, 3600)), 'sequenceID': 1, 'sequenceRemoteControllable': True, 'isStoppable': True, 'earliestStartTime': datetime.datetime(2021, 12, 8, 13, 6, 48, tzinfo=tzoffset(None, 3600)), 'activeSlotNumber': 1, 'startTime': datetime.datetime(2021, 12, 8, 13, 6, 48, tzinfo=tzoffset(None, 3600)), 'endTime': datetime.datetime(2021, 12, 8, 13, 6, 48, tzinfo=tzoffset(None, 3600)), 'state': 'scheduled', 'sequence': {datetime.timedelta(0): 25.906143089083542}}
{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence>', 'valueMax_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 'measurementMax_1': '<https://saref.etsi.org/core/Measurement>', 'latestEndTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 'valueExpectedValue_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 'powerExpectedValue_1': '<https://saref.etsi.org/saref4ener/PowerExpec

In [107]:
datapoint={'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence>', 'valueMax_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 'measurementMax_1': '<https://saref.etsi.org/core/Measurement>', 'latestEndTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 'valueExpectedValue_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 'powerExpectedValue_1': '<https://saref.etsi.org/saref4ener/PowerExpectedValue>', 'defaultDuration_1': "'00:23:00'^^<http://www.w3.org/2001/XMLSchema#time>", 'sequenceID': "'1'^^<http://www.w3.org/2001/XMLSchema#int>", 'sequenceRemoteControllable': "'false'^^<http://www.w3.org/2001/XMLSchema#boolean>", 'measurementMin_1': '<https://saref.etsi.org/core/Measurement>', 'powerMax_1': '<https://saref.etsi.org/saref4ener/PowerMax>', 'slot_1': '<https://saref.etsi.org/saref4ener/Slot>', 'isStoppable': "'false'^^<http://www.w3.org/2001/XMLSchema#boolean>", 'earliestStartTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 'slotNumber_1': "'1'^^<http://www.w3.org/2001/XMLSchema#int>", 'activeSlotNumber': "'1'^^<http://www.w3.org/2001/XMLSchema#int>", 'powerMin_1': '<https://saref.etsi.org/saref4ener/PowerMin>', 'startTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 'valueMin_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>", 'endTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>", 'state': "'scheduled'^^<http://www.w3.org/2001/XMLSchema#string>", 'measurementExpectedValue_1': '<https://saref.etsi.org/core/Measurement>'}

In [108]:
datapoint

{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence>',
 'valueMax_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>",
 'measurementMax_1': '<https://saref.etsi.org/core/Measurement>',
 'latestEndTime': "'2021-12-08T13:06:48+01:00'^^<http://www.w3.org/2001/XMLSchema#dateTime>",
 'valueExpectedValue_1': "'25.906143089083542'^^<http://www.w3.org/2001/XMLSchema#decimal>",
 'powerExpectedValue_1': '<https://saref.etsi.org/saref4ener/PowerExpectedValue>',
 'defaultDuration_1': "'00:23:00'^^<http://www.w3.org/2001/XMLSchema#time>",
 'sequenceID': "'1'^^<http://www.w3.org/2001/XMLSchema#int>",
 'sequenceRemoteControllable': "'false'^^<http://www.w3.org/2001/XMLSchema#boolean>",
 'measurementMin_1': '<https://saref.etsi.org/core/Measurement>',
 'powerMax_1': '<https://saref.etsi.org/saref4ener/PowerMax>',
 'slot_1': '<https://saref.etsi.org/saref4ener/Slot>',
 'isStoppable': "'false'^^<http://www.w3.org/2001/XMLSchema#boolean>",
 'earliestStartTime': "'2021-1

In [39]:
binding_set = [{'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"8"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/9>',
  'powerMax_1': '<https://example/slot/powerMax/9>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"9"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/9>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/9>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/9>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/9>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/9>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"30"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/6>',
  'powerMax_1': '<https://example/slot/powerMax/6>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"6"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/6>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/6>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/6>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/6>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/6>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"26"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/7>',
  'powerMax_1': '<https://example/slot/powerMax/7>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"7"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/7>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/7>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/7>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/7>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/7>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"2"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/16>',
  'powerMax_1': '<https://example/slot/powerMax/16>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"16"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/16>',
  'valueMax_1': '"20"',
  'measurementMax_1': '<https://example/slot/measurementMax/16>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/16>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/16>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/16>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"1900"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/2>',
  'powerMax_1': '<https://example/slot/powerMax/2>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"2"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"100"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/2>',
  'valueMax_1': '"2500"',
  'measurementMax_1': '<https://example/slot/measurementMax/2>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/2>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/2>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/2>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"26"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/3>',
  'powerMax_1': '<https://example/slot/powerMax/3>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"3"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/3>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/3>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/3>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/3>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/3>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"2"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/12>',
  'powerMax_1': '<https://example/slot/powerMax/12>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"12"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/12>',
  'valueMax_1': '"20"',
  'measurementMax_1': '<https://example/slot/measurementMax/12>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/12>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/12>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/12>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"2"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/14>',
  'powerMax_1': '<https://example/slot/powerMax/14>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"14"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/14>',
  'valueMax_1': '"20"',
  'measurementMax_1': '<https://example/slot/measurementMax/14>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/14>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/14>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/14>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"28"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/8>',
  'powerMax_1': '<https://example/slot/powerMax/8>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"8"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/8>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/8>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/8>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/8>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/8>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"26"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/4>',
  'powerMax_1': '<https://example/slot/powerMax/4>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"4"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/4>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/4>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/4>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/4>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/4>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"10"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/11>',
  'powerMax_1': '<https://example/slot/powerMax/11>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"11"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/11>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/11>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/11>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/11>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/11>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"10"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/1>',
  'powerMax_1': '<https://example/slot/powerMax/1>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"1"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/1>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/1>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/1>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/1>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/1>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"1900"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/10>',
  'powerMax_1': '<https://example/slot/powerMax/10>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"10"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"100"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/10>',
  'valueMax_1': '"2500"',
  'measurementMax_1': '<https://example/slot/measurementMax/10>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/10>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/10>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/10>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"2"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/15>',
  'powerMax_1': '<https://example/slot/powerMax/15>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"15"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/15>',
  'valueMax_1': '"20"',
  'measurementMax_1': '<https://example/slot/measurementMax/15>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/15>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/15>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/15>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"2"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/13>',
  'powerMax_1': '<https://example/slot/powerMax/13>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"13"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/13>',
  'valueMax_1': '"20"',
  'measurementMax_1': '<https://example/slot/measurementMax/13>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/13>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/13>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/13>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'},
 {'powerSequence': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268>',
  'valueExpectedValue_1': '"28"',
  'deviceId': '<http://example.org/spine-ssa/devices/012100518398009540>',
  'associatedDevice': '<http://example.org/spine-ssa/devices/012100518398009540/powerSequences/187955268/associatedDevice>',
  'sequenceRemoteControllable': '"true"',
  'slot_1': '<https://example/slot/5>',
  'powerMax_1': '<https://example/slot/powerMax/5>',
  'earliestStartTime': '"2023-06-28T12:27:48.727283187Z"',
  'isPausable': '"false"',
  'slotNumber_1': '"5"',
  'activeSlotNumber': '"0"',
  'startTime': '"2023-06-28T15:32:48.727283187Z"',
  'valueMin_1': '"1"',
  'state': '"scheduled"',
  'measurementExpectedValue_1': '<https://example/slot/measurementExpectedValue/5>',
  'valueMax_1': '"100"',
  'measurementMax_1': '<https://example/slot/measurementMax/5>',
  'latestEndTime': '"2023-06-28T19:27:48.727283187Z"',
  'powerExpectedValue_1': '<https://example/slot/powerExpectedValue/5>',
  'defaultDuration_1': '"00:14:41"',
  'sequenceID': '"187955268"',
  'measurementMin_1': '<https://example/slot/measurementMin/5>',
  'isStoppable': '"false"',
  'powerMin_1': '<https://example/slot/powerMin/5>',
  'endTime': '"2023-06-28T19:27:48.727283187Z"'}]

In [40]:
def parse_bsh_miele(datapoint: dict):
    """
    point contains string values, to be converted to the correct type, depending on the key

    returns
    DateTime object for dateTime literals
    float object for decimal literals
    """
    ints = ['slotNumber_1', 'activeSlotNumber']
    floats = ['valueMin_1', 'valueExpectedValue_1', 'valueMax_1']
    booleans = ['sequenceRemoteControllable', 'isPausable', 'isStoppable']
    strings = ['sequenceID', 'state', 'associatedDevice']
    timestamps = ['earliestStartTime', 'startTime', 'latestEndTime', 'endTime']
    hours = ['defaultDuration_1']
    last_value_url = ['powerSequence', 'deviceId', 'slot_1', 'powerMax_1', 'measurementExpectedValue_1',
                     'measurementMax_1', 'powerExpectedValue_1', 'powerExpectedValue_1', 'measurementMin_1',
                     'powerMin_1']
    
    def parse_time(t):
        time_split = t.split(':')
        return datetime.timedelta(hours=int(time_split[0]),
                                  minutes=int(time_split[1]),
                                  seconds=int(time_split[2]))
    d = {}
    for k,v in datapoint.items():
        v = v.replace("'", '').replace('"', '')
        if k in ints:
            d[k] = int(float(v))
        elif k in floats:
            d[k] = float(v)
        elif k in booleans:
            d[k] = bool(v)
        elif k in booleans:
            d[k] = bool(v)
        elif k in strings:
            d[k] = str(v)
        elif k in timestamps:
            d[k] = parse(v)
        elif k in hours:
            d[k] = parse_time(v)
        elif k in last_value_url:
            v = v.split('/')[-1]
            if '_1' in k:
                v = int(v.replace(">", ''))
            d[k] = v
    
    return d

In [43]:
device_info = {}
profile = {}
device_related_keys = ['powerSequence', 'deviceId', 'associatedDevice', 'sequenceRemoteControllable',
                       'isPausable', 'earliestStartTime', 'activeSlotNumber', 'startTime', 'state', 
                       'latestEndTime', 'sequenceID', 'isStoppable', 'endTime']
profile_related_keys = [k for k in binding_set[0].keys() if any(c.isdigit() for c in k)]

for datapoint in binding_set: 
    # parse the whole datapoint
    datapoint = parse_bsh_miele(datapoint)
    slot_number = datapoint['slotNumber_1']
    profile[slot_number] = (datapoint[f'defaultDuration_1'], datapoint[f'valueExpectedValue_1'])

profile = dict(sorted(profile.items()))
d = {}
prev_time = datetime.timedelta(0)
for v in profile.values():
    d[prev_time] = v[1]
    prev_time += v[0]
#d = {k - min(d.keys()): v for k, v in d.items()}
d[prev_time] = 0.0
    
#profile = {k: v for k, v in zip([datetime.timedelta(0)] + list(d.keys()), list(d.values()) + [0.0])}

In [44]:
d

{datetime.timedelta(0): 10.0,
 datetime.timedelta(seconds=881): 1900.0,
 datetime.timedelta(seconds=1762): 26.0,
 datetime.timedelta(seconds=2643): 26.0,
 datetime.timedelta(seconds=3524): 28.0,
 datetime.timedelta(seconds=4405): 30.0,
 datetime.timedelta(seconds=5286): 26.0,
 datetime.timedelta(seconds=6167): 28.0,
 datetime.timedelta(seconds=7048): 8.0,
 datetime.timedelta(seconds=7929): 1900.0,
 datetime.timedelta(seconds=8810): 10.0,
 datetime.timedelta(seconds=9691): 2.0,
 datetime.timedelta(seconds=10572): 2.0,
 datetime.timedelta(seconds=11453): 2.0,
 datetime.timedelta(seconds=12334): 2.0,
 datetime.timedelta(seconds=13215): 2.0,
 datetime.timedelta(seconds=14096): 0.0}

## Whirlpool

In [ ]:
[{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 'valueMax': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 'valueExpectedValue': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 'measurementMin': '<https://example/slot/measurementMin/4>', 'powerExpectedValue': '<https://example/slot/powerExpectedValue/4>', 'slot': '<https://example/slot/4>', 'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/4>', 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMin': '<https://example/slot/powerMin/4>', 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'slotNumber': '"4"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'defaultDuration': '"00:04:00"^^<http://www.w3.org/2001/XMLSchema#string>', 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMax': '<https://example/slot/powerMax/4>', 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'measurementMax': '<https://example/slot/measurementMax/4>', 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}, {'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 'valueMax': '"1000"^^<http://www.w3.org/2001/XMLSchema#integer>', 'valueExpectedValue': '"1000"^^<http://www.w3.org/2001/XMLSchema#integer>', 'measurementMin': '<https://example/slot/measurementMin/3>', 'powerExpectedValue': '<https://example/slot/powerExpectedValue/3>', 'slot': '<https://example/slot/3>', 'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/3>', 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMin': '<https://example/slot/powerMin/3>', 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'slotNumber': '"3"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'defaultDuration': '"00:03:00"^^<http://www.w3.org/2001/XMLSchema#string>', 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMax': '<https://example/slot/powerMax/3>', 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'measurementMax': '<https://example/slot/measurementMax/3>', 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}, {'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 'valueMax': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 'valueExpectedValue': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 'measurementMin': '<https://example/slot/measurementMin/1>', 'powerExpectedValue': '<https://example/slot/powerExpectedValue/1>', 'slot': '<https://example/slot/1>', 'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/1>', 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMin': '<https://example/slot/powerMin/1>', 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'slotNumber': '"1"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'defaultDuration': '"00:09:00"^^<http://www.w3.org/2001/XMLSchema#string>', 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMax': '<https://example/slot/powerMax/1>', 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'measurementMax': '<https://example/slot/measurementMax/1>', 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}, {'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 'valueMax': '"2600"^^<http://www.w3.org/2001/XMLSchema#integer>', 'valueExpectedValue': '"2600"^^<http://www.w3.org/2001/XMLSchema#integer>', 'measurementMin': '<https://example/slot/measurementMin/2>', 'powerExpectedValue': '<https://example/slot/powerExpectedValue/2>', 'slot': '<https://example/slot/2>', 'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/2>', 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMin': '<https://example/slot/powerMin/2>', 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'slotNumber': '"2"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'defaultDuration': '"00:05:00"^^<http://www.w3.org/2001/XMLSchema#string>', 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMax': '<https://example/slot/powerMax/2>', 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'measurementMax': '<https://example/slot/measurementMax/2>', 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}]

In [47]:
binding_set = [{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 
                'valueMax': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 
                'valueExpectedValue': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 
                'measurementMin': '<https://example/slot/measurementMin/4>', 
                'powerExpectedValue': '<https://example/slot/powerExpectedValue/4>', 
                'slot': '<https://example/slot/4>', 
                'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 
                'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 
                'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 
                'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 
                'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
                'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/4>', 
                'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 
                'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
                'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 
                'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 
                'powerMin': '<https://example/slot/powerMin/4>', 
                'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
                'slotNumber': '"4"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 
                'defaultDuration': '"00:04:00"^^<http://www.w3.org/2001/XMLSchema#string>', 
                'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 
                'powerMax': '<https://example/slot/powerMax/4>', 
                'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 
                'measurementMax': '<https://example/slot/measurementMax/4>', 
                'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
                'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}, 
               {'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 
                'valueMax': '"1000"^^<http://www.w3.org/2001/XMLSchema#integer>', 
                'valueExpectedValue': '"1000"^^<http://www.w3.org/2001/XMLSchema#integer>', 
                'measurementMin': '<https://example/slot/measurementMin/3>', 
                'powerExpectedValue': '<https://example/slot/powerExpectedValue/3>', 
                'slot': '<https://example/slot/3>', 
                'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/3>', 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMin': '<https://example/slot/powerMin/3>', 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'slotNumber': '"3"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'defaultDuration': '"00:03:00"^^<http://www.w3.org/2001/XMLSchema#string>', 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMax': '<https://example/slot/powerMax/3>', 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'measurementMax': '<https://example/slot/measurementMax/3>', 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}, {'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 'valueMax': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 'valueExpectedValue': '"400"^^<http://www.w3.org/2001/XMLSchema#integer>', 'measurementMin': '<https://example/slot/measurementMin/1>', 'powerExpectedValue': '<https://example/slot/powerExpectedValue/1>', 'slot': '<https://example/slot/1>', 'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/1>', 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMin': '<https://example/slot/powerMin/1>', 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'slotNumber': '"1"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'defaultDuration': '"00:09:00"^^<http://www.w3.org/2001/XMLSchema#string>', 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMax': '<https://example/slot/powerMax/1>', 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'measurementMax': '<https://example/slot/measurementMax/1>', 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}, {'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 'valueMax': '"2600"^^<http://www.w3.org/2001/XMLSchema#integer>', 'valueExpectedValue': '"2600"^^<http://www.w3.org/2001/XMLSchema#integer>', 'measurementMin': '<https://example/slot/measurementMin/2>', 'powerExpectedValue': '<https://example/slot/powerExpectedValue/2>', 'slot': '<https://example/slot/2>', 'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 'valueMin': '"0"^^<http://www.w3.org/2001/XMLSchema#integer>', 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'measurementExpectedValue': '<https://example/slot/measurementExpectedValue/2>', 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMin': '<https://example/slot/powerMin/2>', 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 'slotNumber': '"2"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 'defaultDuration': '"00:05:00"^^<http://www.w3.org/2001/XMLSchema#string>', 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 'powerMax': '<https://example/slot/powerMax/2>', 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 'measurementMax': '<https://example/slot/measurementMax/2>', 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
               'valueSource': '"calculatedValue"^^<http://www.w3.org/2001/XMLSchema#string>'}]

In [48]:
device_info = {}
profile = {}
device_related_keys = ['sequenceRemoteControllable', 'deviceAddress', 'isPausable',
                       'earliestStartTime', 'activeSlotNumber', 'startTime', 'state', 'taskID',
                       'latestEndTime', 'sequenceID', 'isStoppable', 'endTime']
# Get values that are general to the sequence, not for a specific slot
device_info.update({k: parse_literal(v) for k, v in binding_set[0].items() if k in device_related_keys})
for datapoint in binding_set:
    slot_number = parse_literal(datapoint['slotNumber'])
    profile[slot_number] = (parse_literal(datapoint[f'defaultDuration'], parse_as='time'),
                            parse_literal(datapoint[f'valueExpectedValue']))
profile = dict(sorted(profile.items()))
d = {}
prev_time = datetime.timedelta(0)
for v in profile.values():
    d[prev_time] = v[1]
    prev_time = prev_time + v[0]
d[prev_time] = 0.0
device_info.update({'sequence': d})

In [49]:
d

{datetime.timedelta(0): 400,
 datetime.timedelta(seconds=540): 2600,
 datetime.timedelta(seconds=840): 1000,
 datetime.timedelta(seconds=1020): 400,
 datetime.timedelta(seconds=1260): 0.0}

In [30]:
profile

{1: (datetime.timedelta(seconds=540), 400),
 2: (datetime.timedelta(seconds=300), 2600),
 3: (datetime.timedelta(seconds=180), 1000),
 4: (datetime.timedelta(seconds=240), 400)}

In [8]:
profile

{datetime.timedelta(0): 400,
 datetime.timedelta(seconds=540): 2600,
 datetime.timedelta(seconds=300): 1000,
 datetime.timedelta(seconds=180): 400,
 datetime.timedelta(seconds=240): 0.0}

?powerSequence <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://saref.etsi.org/saref4ener/PowerSequence> . 
?powerSequence <https://saref.etsi.org/saref4ener/deviceCode> ?deviceAddress . 
?powerSequence <https://saref.etsi.org/saref4ener/sequenceID> ?sequenceID .

[{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>',
  'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>',
  'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>'}]

In [ ]:
{k: v for k, v in binding_set.items() if k in complete_set_keys}

In [52]:
complete_set.keys()

dict_keys(['powerSequence', 'deviceAddress', 'sequenceID'])

Sequence data extracted

In [ ]:
{'powerSequence': '<https://saref.etsi.org/saref4ener/PowerSequence/Timer:WPR4MV79BF943:1658503794187:CNC>', 
 'sequenceRemoteControllable': '"true"^^<http://www.w3.org/2001/XMLSchema#boolean>', 
 'deviceAddress': '"WPR4MV79BF943"^^<http://www.w3.org/2001/XMLSchema#string>', 
 'isPausable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 
 'earliestStartTime': '"2022-07-22T15:36:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
 'activeSlotNumber': '"0"^^<http://www.w3.org/2001/XMLSchema#unsignedInt>', 
 'startTime': '"2022-12-24T16:29:33+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
 'state': '"scheduled"^^<http://www.w3.org/2001/XMLSchema#string>', 
 'taskID': '"WashCycleQuickWash15-3"^^<http://www.w3.org/2001/XMLSchema#string>', 
 'latestEndTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
 'sequenceID': '"Timer:WPR4MV79BF943:1658503794187:CNC"^^<http://www.w3.org/2001/XMLSchema#string>', 
 'isStoppable': '"false"^^<http://www.w3.org/2001/XMLSchema#boolean>', 
 'endTime': '"2022-12-24T16:29:48+0000"^^<http://www.w3.org/2001/XMLSchema#dateTime>', 
 'sequence': {datetime.timedelta(0): 400, 
              datetime.timedelta(seconds=540): 2600, 
              datetime.timedelta(seconds=300): 1000, 
              datetime.timedelta(seconds=180): 400, 
              datetime.timedelta(seconds=240): 0.0}
}